In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from bs4 import BeautifulSoup
from llama_cpp import Llama

/Users/zostaw/.pyenv/versions/3.12.2/envs/ai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

In [3]:
model_path="./models/openhermes-2.5-mistral-7b-16k.Q5_K_M.gguf"
llm = Llama(
        model_path=model_path,
        n_ctx=4096,
        temperature=0.7,
        verbose=False,
        n_gpu_layers=1,
    )


## Context 1

In [4]:
context1 = """
Beautiful Soup is a Python library for parsing structured data. It allows you to interact with HTML in a similar way to how you interact with a web page using developer tools. The library exposes a couple of intuitive functions you can use to explore the HTML you received.

"""

## Context 2

In [5]:
context2 = """
```python
>>> from bs4 import BeautifulSoup
>>> soup = BeautifulSoup("<p>Some<b>bad<i>HTML")
>>> print(soup.prettify())
<html>
 <body>
  <p>
   Some
   <b>
    bad
    <i>
     HTML
    </i>
   </b>
  </p>
 </body>
</html>
>>> soup.find(text="bad")
'bad'
>>> soup.i
<i>HTML</i>
#
>>> soup = BeautifulSoup("<tag1>Some<tag2/>bad<tag3>XML", "xml")
#
>>> print(soup.prettify())
<?xml version="1.0" encoding="utf-8"?>
<tag1>
 Some
 <tag2/>
 bad
 <tag3>
  XML
 </tag3>
</tag1>
```
"""

In [6]:
question = "How to use BeautifulSoup?"

In [19]:
prompt = f"""Answer this question with the help of provided contexts. Before responding stop and think about each of them, find how they related to each other or if there are important connections, then provide answer. If you find that some of the provided contexts does not matter, ignore it.\n
## CONTEXT 1
{context1}
## CONTEXT 2
{context2}
## QUESTION
{question}
## ANSWER
"""

In [20]:
print(prompt)

Answer this question with the help of provided contexts. Before responding stop and think about each of them, find how they related to each other or if there are important connections, then provide answer. If you find that some of the provided contexts does not matter, ignore it.

## CONTEXT 1 

Beautiful Soup is a Python library for parsing structured data. It allows you to interact with HTML in a similar way to how you interact with a web page using developer tools. The library exposes a couple of intuitive functions you can use to explore the HTML you received.

 
## CONTEXT 2 

```python
>>> from bs4 import BeautifulSoup
>>> soup = BeautifulSoup("<p>Some<b>bad<i>HTML")
>>> print(soup.prettify())
<html>
 <body>
  <p>
   Some
   <b>
    bad
    <i>
     HTML
    </i>
   </b>
  </p>
 </body>
</html>
>>> soup.find(text="bad")
'bad'
>>> soup.i
<i>HTML</i>
#
>>> soup = BeautifulSoup("<tag1>Some<tag2/>bad<tag3>XML", "xml")
#
>>> print(soup.prettify())
<?xml version="1.0" encoding="utf-8"?

In [21]:
output = llm(
      prompt, # Prompt
      max_tokens=2000, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=False # Echo the prompt back in the output
)

In [22]:

output["choices"][0]["text"]

"In order to use BeautifulSoup, you need to first import the library using `from bs4 import BeautifulSoup`. Then create a `BeautifulSoup` object, passing in your HTML and an optional parser (usually `'html.parser'`) as arguments, like so: `soup = BeautifulSoup(markup, 'html.parser')`. This object can be used to perform various operations on your HTML, such as finding tags and their text content, prettifying your HTML, etc."